In [6]:
import copy
import numpy as np
import math
import pandas as pd
from scipy.io import loadmat
import warnings
warnings.filterwarnings('ignore')
import random

## Functions

In [7]:
def standardization(data,dataset):
    
    #正規化したデータを格納するリスト
    standard_data = []
    
    for i in range(dataset[0]):
        
        #最大値，最小値の取得
        max_value = max(data[i])
        min_value = min(data[i])
        
        #正規化したデータを仮に格納しておくリスト
        temp =[]
        
        #データセットの数だけ正規化を行う
        for j in range(dataset[1]):
            temp.append((data[i][j] - min_value)/(max_value - min_value))
        
        standard_data.append(temp)
            
    return standard_data   


def discretization(data,dataset,level):
    
    #離散化したデータを格納するリスト
    discretiza_data = []

    #データセットの数だけ離散化を行う
    for i in range(dataset[0]):
        discretiza_data.append(pd.cut(data[i],level,labels=False))
        
    return discretiza_data

In [8]:
def emma_join(episode, origin_dic, key_list, minsup):
    key_length = len(list(episode.keys())[0])
    
    #Serial Extensionされたepisode(長さ2以上)に対して、その後にfrequent eventが続くかどうかを探索
    for key_idx in range(len(key_list)):
        #bound_listの各要素(出現位置)+key_length-1でend_time(直後に続くイベントの出現位置)が出せることを利用
        bool_list = list(map(lambda comp_value: comp_value+key_length in list(map(lambda value: value, origin_dic[key_list[key_idx]])), list(np.array(list(episode.values())).flatten())))
        sup = bool_list.count(True)

        if sup >= minsup:
            key = tuple(list(list(episode.keys())[0])+[key_list[key_idx]])
            extension_ep = {}
            extension_ep.setdefault(key, []).append([value for value, bool_data in zip(list(np.array(list(episode.values())).flatten()), bool_list) if bool_data == True])
            extension_ep[key] = list(np.array(list(extension_ep.values())).flatten())
            compositte_episode.update(extension_ep) #Serial extensionしたepisodeを保存
            emma_join(extension_ep, origin_dic, key_list, minsup)

In [9]:
file_num = 2
mat_data = loadmat(f'./dataset/dataset_for_accuracy/sample{file_num}.mat')
# mat_data = loadmat(f'./dataset/dataset_for_accuracy/sample{file_num}_var001.mat')
data = mat_data['sample']
data_size = data.shape

standarded_data = standardization(data, data_size)

level = 50
discretized_data = discretization(standarded_data, data_size, level)

sequence = discretized_data[2].copy()

#paramerter
minsup = 5
winlen = 5

global compositte_episode
count_of_support = {}

#step1-3
[count_of_support.setdefault(sequence[i], []).append(i) for i in range(data_size[1])] #eventのsupportを数える
occurr = {key:value for key, value in count_of_support.items() if len(value) >= minsup} #sup >= minsup のeventだけ保持
compositte_episode = occurr.copy()
key_list = list(occurr.keys())

#Serial Extension
for key_idx in range(len(key_list)):
    for key_idx_2 in range(len(key_list)):
        #sup >= minsup then continue, else break
        bool_list = list(map(lambda comp_value: comp_value in occurr[key_list[key_idx_2]], list(map(lambda value: value+1, occurr[key_list[key_idx]]))))
        sup = bool_list.count(True)
        
        if sup >= minsup:
            extension_ep = {}
            key = (key_list[key_idx], key_list[key_idx_2])
            extension_ep.setdefault(key, []).append([value for value, bool_data in zip(occurr[key_list[key_idx]], bool_list) if bool_data == True])
            extension_ep[key] = list(np.array(list(extension_ep.values())).flatten())
#             print(extension_ep)
            compositte_episode.update(extension_ep)
            emma_join(extension_ep, occurr, key_list, minsup)
        
compositte_episode

{29: [0, 84, 87, 98, 304, 365, 372, 590, 699, 771, 972],
 37: [1,
  80,
  103,
  168,
  204,
  265,
  270,
  308,
  311,
  386,
  406,
  411,
  485,
  509,
  669,
  693,
  768,
  776,
  793,
  795,
  866],
 27: [2, 212, 290, 367, 381, 496, 507, 579, 582, 585, 593, 600, 678, 981],
 1: [3, 81, 484, 696, 775, 876, 967],
 12: [5, 113, 201, 310, 398, 568, 697],
 24: [6, 10, 82, 305, 467, 491, 613, 688, 800, 889, 971],
 8: [7, 75, 110, 182, 189, 191, 412, 508, 589, 883, 898, 900],
 48: [8, 86, 199, 268, 287, 402, 580, 595, 674, 880, 968],
 15: [9,
  17,
  27,
  37,
  47,
  57,
  105,
  117,
  127,
  137,
  147,
  157,
  185,
  217,
  227,
  237,
  247,
  257,
  313,
  317,
  327,
  337,
  347,
  357,
  397,
  417,
  427,
  437,
  447,
  457,
  490,
  517,
  527,
  537,
  547,
  557,
  617,
  627,
  637,
  647,
  657,
  717,
  727,
  737,
  747,
  757,
  787,
  801,
  817,
  827,
  837,
  847,
  857,
  896,
  917,
  927,
  937,
  947,
  957,
  976],
 44: [11, 498, 513, 675, 703, 895, 905],
 3

## unit test

In [18]:
#テストデータの作成
pattern = [1,2,3,4,5]
pattern_2 = [100, 200, 300]
sequence = [random.randint(0,49) for _ in range(200)]

idx = 0
while(idx<=199):
    if idx % 50 == 0:
        sequence[idx:idx+len(pattern_2)] = pattern_2
        idx += len(pattern_2)
    if idx % 15 == 0:
        sequence[idx:idx+len(pattern)] = pattern
        idx += len(pattern)
    else:
        idx += 1

In [19]:
#paramerter
data_size = [1, len(sequence)]
minsup = 3
winlen = 5

global compositte_episode
count_of_support = {}

#step1-3
[count_of_support.setdefault(sequence[i], []).append(i) for i in range(data_size[1])] #eventのsupportを数える
occurr = {key:value for key, value in count_of_support.items() if len(value) >= minsup} #sup >= minsup のeventだけ保持
compositte_episode = occurr.copy()
key_list = list(occurr.keys())

#Serial Extension
for key_idx in range(len(key_list)):
    for key_idx_2 in range(len(key_list)):
        #sup >= minsup then continue, else break
        bool_list = list(map(lambda comp_value: comp_value in occurr[key_list[key_idx_2]], list(map(lambda value: value+1, occurr[key_list[key_idx]]))))
        sup = bool_list.count(True)
            
        if sup >= minsup:
            extension_ep = {}
            key = (key_list[key_idx], key_list[key_idx_2])
            extension_ep.setdefault(key, []).append([value for value, bool_data in zip(occurr[key_list[key_idx]], bool_list) if bool_data == True])
            extension_ep[key] = list(np.array(list(extension_ep.values())).flatten())
            compositte_episode.update(extension_ep)
            emma_join(extension_ep, occurr, key_list, minsup)
        
compositte_episode

{100: [0, 50, 100, 150],
 200: [1, 51, 101, 151],
 300: [2, 52, 102, 152],
 3: [3,
  12,
  17,
  32,
  47,
  54,
  62,
  74,
  77,
  92,
  107,
  122,
  133,
  137,
  143,
  157,
  167,
  173,
  182,
  197],
 33: [4, 23, 98, 164, 188],
 20: [5, 28, 80],
 46: [6, 13, 56, 58, 87, 159, 194],
 18: [7, 40, 99],
 1: [10,
  15,
  30,
  44,
  45,
  60,
  75,
  90,
  105,
  120,
  125,
  132,
  135,
  165,
  179,
  180,
  195],
 2: [16, 31, 46, 57, 61, 76, 91, 106, 121, 136, 166, 181, 196],
 4: [18, 29, 33, 48, 63, 78, 93, 108, 123, 138, 148, 168, 183, 198],
 5: [19, 34, 49, 53, 64, 79, 94, 109, 124, 139, 161, 169, 184, 199],
 8: [20, 70, 154, 176],
 47: [25, 38, 73, 178],
 38: [26, 41, 88, 111, 114, 118, 144, 186],
 40: [27, 36, 113],
 16: [35, 96, 145, 187, 190],
 14: [37, 39, 153, 189],
 34: [43, 67, 155, 160],
 35: [59, 104, 185],
 28: [65, 84, 172],
 17: [68, 156, 162],
 32: [95, 163, 175],
 6: [97, 126, 130],
 44: [112, 117, 141, 142, 177, 191, 192],
 30: [131, 174, 193],
 (100, 200): [0,

In [24]:
file_num = 2
mat_data = loadmat(f'./dataset/dataset_for_accuracy/sample{file_num}.mat')
# mat_data = loadmat(f'./dataset/dataset_for_accuracy/sample{file_num}_var001.mat')
data = mat_data['sample']
data_size = data.shape

standarded_data = standardization(data, data_size)

level = 50
discretized_data = discretization(standarded_data, data_size, level)

sequence = discretized_data[1].copy()
#paramerter
minsup = 5
winlen = 5

global compositte_episode
count_of_support = {}

#step1-3
[count_of_support.setdefault(sequence[i], []).append(i) for i in range(data_size[1])] #eventのsupportを数える
occurr = {key:value for key, value in count_of_support.items() if len(value) >= minsup} #sup >= minsup のeventだけ保持
compositte_episode = occurr.copy()
key_list = list(occurr.keys())

#Serial Extension
for key_idx in range(len(key_list)-1): 
    #sup >= minsup then continue, else break
    bool_list = list(map(lambda comp_value: comp_value in occurr[key_list[key_idx+1]], list(map(lambda value: value+1, occurr[key_list[key_idx]]))))
    sup = bool_list.count(True)
    print(f'sup:{sup}')
    if sup >= minsup:
        extension_ep = {}
        extension_ep.setdefault((key_list[key_idx], key_list[key_idx+1]), []).append([value for value, bool_data in zip(occurr[key_list[key_idx]], bool_list) if bool_data == True])
        extension_ep[(key_list[key_idx], key_list[key_idx+1])] = list(np.array(extension_ep[(key_list[key_idx],key_list[key_idx+1])]).flatten())
        #Serial Extension
        emma_join(extension_ep, occurr, key_list, minsup)

sup:1
sup:1
sup:1
sup:1
sup:1
sup:1
sup:1
sup:1
sup:1
sup:1
sup:1
sup:0
sup:1
sup:1
sup:4
sup:1
sup:0
sup:1
sup:1
sup:0
sup:2
sup:1
sup:0
sup:1
sup:0
sup:0
sup:0
sup:0
sup:1
sup:0
sup:0
sup:1
sup:0
sup:0
sup:0
sup:0
sup:1
sup:1
sup:0
sup:0
sup:0
sup:0
sup:0
sup:0
sup:0
sup:0
sup:0
sup:0
sup:0
